In [1]:
import pandas as pd

In [3]:
pop_den = pd.read_csv('Internet_Master_Plan__Adoption_and_Infrastructure_Data_by_Neighborhood.csv')

In [5]:
pop_den = pop_den[['Neighborhood Tabulation Area Name (NTA NAME)', 'Population Density (per Sq. Mi.)']]

In [7]:
pop_den.rename(columns={'Neighborhood Tabulation Area Name (NTA NAME)':'name', 'Population Density (per Sq. Mi.)':'pop_density'}, inplace=True)

C:\Users\eric_\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
pop_den['name'] = pop_den['name'].str.split('-')

<ipython-input-9-1968bb0f6f20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_den['name'] = pop_den['name'].str.split('-')


In [11]:
pop_den1 = pop_den.explode('name')

In [12]:
pop_den1

,name,pop_density
0,Brooklyn Heights,65994.44
0,Cobble Hill,65994.44
1,Sheepshead Bay,28707.93
1,Gerritsen Beach,28707.93
1,Manhattan Beach,28707.93
...,...,...
193,Great Kills,12998.13
194,park,0.00
194,cemetery,0.00
194,etc,0.00


In [18]:
pop_den1.sort_values(by='name', inplace=True)

In [34]:
pop_den1.reset_index(inplace=True, level=0)

In [37]:
pop_den1.drop(columns=['level_0', 'index'], inplace=True)

In [13]:
# function to fully print entire dataframe for visual
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [110]:
print_full(pop_den1)

                          name  pop_density
0                     Allerton     27278.07
1                     Annadale      5889.53
2                Arden Heights     14854.70
3                    Arlington     10275.86
4                     Arrochar     11168.03
5                      Arverne     19218.47
6                      Astoria     52548.94
7                   Auburndale     16455.28
8                   Bath Beach     45378.38
9            Battery Park City     68332.35
10                   Bay Ridge     32735.27
11                 Bay Terrace     13246.99
12                  Baychester     26643.75
13                   Bayswater     29771.65
14          Bedford Stuyvesant     67585.47
15                Bedford Park    108062.96
16                Belle Harbor      7844.26
17                   Bellerose     13961.11
18                     Belmont     59106.12
19                 Bensonhurst     51241.41
20                 Bensonhurst     53825.75
21                Bergen Beach  

In [108]:
# manually update all discrepancies by manually cross-referencing between uber data names and nyc_geo data names
pop_den1.at[283, 'name'] = 'Westchester Square'

In [109]:
# remove columns that do not exist in original nyc_geo data
indexname = pop_den1[pop_den1['name'] == 'op City'].index
pop_den1.drop(indexname, inplace=True)

In [44]:
# manually update all discrepancies by manually cross-referencing between uber data names and nyc_geo data names
pop_den1.at[29, 'pop_density'] = 14730.34

In [111]:
pop_den2 = pop_den1.groupby(by='name', as_index=False).mean()

In [114]:
nyc_data = pd.read_csv('nyc_data.csv')

In [119]:
nyc_name = nyc_data[['name', 'adj_pop']]

In [122]:
nyc_den = nyc_name.merge(pop_den2, how='left', on='name')

In [123]:
nyc_den.drop(columns='adj_pop', inplace=True)

In [125]:
nyc_den.isnull().sum()

name            0
pop_density    50
dtype: int64

In [200]:
print_full(nyc_den)

                          name    pop_density
0                     Allerton   27278.070000
1                     Annadale    5889.530000
2                Arden Heights   14854.700000
3                    Arlington   10275.860000
4                     Arrochar   11168.030000
5                      Arverne   19218.470000
6                      Astoria   52548.940000
7              Astoria Heights   42310.710000
8                   Auburndale   16455.280000
9                   Bath Beach   45378.380000
10           Battery Park City   68332.350000
11                   Bay Ridge   32735.270000
12                 Bay Terrace   13246.990000
13                  Baychester   26643.750000
14                     Bayside   14730.340000
15                   Bayswater   29771.650000
16                Bedford Park  108062.960000
17          Bedford Stuyvesant   67585.470000
18                  Beechhurst    9663.000000
19                    Bellaire   14144.000000
20                Belle Harbor    

In [198]:
# manually update all discrepancies by manually cross-referencing between uber data names and nyc_geo data names
nyc_den.at[294, 'pop_density'] = 5631.01

In [199]:
nyc_den.isnull().sum()

name           0
pop_density    0
dtype: int64

In [201]:
nyc_den.to_csv('nyc_pop_density.csv', index=False)